<a href="https://colab.research.google.com/github/TarunTejaYaram/Openl3---classification/blob/master/fen%C3%AAtre_texture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet CENSE

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp /content/drive/My\ Drive/project/scenes_stereo.zip .
!unzip scenes_stereo.zip

In [0]:
!pip3 install openl3
!pip3 install soundfile
!pip3 uninstall tensorflow-gpu
!pip3 install tensorflow-gpu==1.14

Uninstalling tensorflow-gpu-1.14.0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_gpu-1.14.0.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/audio/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/autodiff/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/autograph/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/autograph/experimental/__init__.py
    /usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/bitwise/__init__.py
    /usr/local/lib/python3.6/dist-packages/t

In [0]:
import os
import numpy as np 
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import openl3
import soundfile as sf

Using TensorFlow backend.


ImportError: ignored

## 1. Preprocessing

In [0]:
#set directory for source files 
data_dir = './scenes_stereo/'

audio_files = glob(data_dir + '*.wav')
print('file size:',len(audio_files))

file size: 100


In [0]:
data_list = glob(data_dir+"/*")
data = []

for name in ['bus','busystreet','office','openairmarket','park','quietstreet','restaurant','supermarket','tube','tubestation']:
    for k in range(1,10):
      audio, sr = sf.read(data_dir+name + '0'+str(k)+'.wav')
      data.append(openl3.get_audio_embedding(audio, sr, content_type="env", input_repr="linear", embedding_size=512))
    audio, sr = sf.read(data_dir+name + '10.wav')
    data.append(openl3.get_audio_embedding(audio, sr, content_type="env", input_repr="linear", embedding_size=512))

In [0]:
data[0].shape

(40, 1292)

In [0]:
np.mean(data[0],axis=1).shape

(40,)

In [0]:
labels = [] # Labels des échantillons
for k in range(10):
    labels = labels+[k]*10
    
dict_labels = {'bus':0,'busystreet':1,'office':2,'openairmarket':3,'park':4,'quietstreet':5,'restaurant':6,'supermarket':7,'tube':8,'tubestation':9} 

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# define the keras model
model = Sequential()
model.add(Dense(data[0].shape, input_dim= 512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(audio, labels, epochs=150, batch_size=10)